# Annotation model evaluation

In [1]:
import tensorflow as tf
import h5py
import numpy as np
from sklearn.metrics import average_precision_score
import rna_model
import torch
import h5py
from torch.utils.data import DataLoader
from tqdm import tqdm
import glob


In [2]:
data_dir = './data/annotation/2000_1000_UTR.h5'
test_loader = DataLoader(rna_model.h5dataset(data_dir,'test')
                    ,num_workers=4,pin_memory=True,batch_size = 128)
f = h5py.File(data_dir,'r')
y_test = f['y_test'][()]
print(y_test.shape)
f.close()

(38555, 2000, 4)


In [3]:
#pytorch load model
model_dir = glob.glob('/home/amber/multitask_RNA/wandb/run-*zai2tsbz/files/best_model.ckpt')[0]
#model = rna_model.Residual_annotation((4,3000),(3,2000),32,0.01)
model = rna_model.Splice_AI_2K((4,3000),(3,2000),32,0.001)
model.load_state_dict(torch.load(model_dir)['state_dict'])

<All keys matched successfully>

In [4]:
%time
pred_list = []
for i, batch in enumerate(tqdm(test_loader)):
        pred =model.predict_step(batch[0],i)
        pred_list.extend(torch.nn.functional.softmax(pred,dim = 1).detach().numpy())
pred_list = np.array(pred_list)

CPU times: user 1 µs, sys: 2 µs, total: 3 µs
Wall time: 6.91 µs


100%|████████████████████████████████████████| 302/302 [05:59<00:00,  1.19s/it]


In [5]:
splice_test= y_test[:,:,:3]
splice_pred = np.swapaxes(np.array(pred_list),1,2)

In [6]:
Y_true_1 = []
Y_true_2 = []
Y_pred_1 = []
Y_pred_2 = []
is_expr = (splice_test.sum(axis=(1,2)) >= 1)
Y_true_1=splice_test[is_expr, :, 1].flatten()
Y_true_2=splice_test[is_expr, :, 2].flatten()
Y_pred_1=splice_pred[is_expr, :, 1].flatten()
Y_pred_2=splice_pred[is_expr, :, 2].flatten()

In [7]:
# task 1 , 2 are donor/acceptor respectively
def print_topl_statistics(y_true, y_pred):
    # Prints the following information: top-kL statistics for k=0.5,1,2,4,
    # auprc, thresholds for k=0.5,1,2,4, number of true splice sites.

    idx_true = np.nonzero(y_true == 1)[0]
    argsorted_y_pred = np.argsort(y_pred)
    sorted_y_pred = np.sort(y_pred)

    topkl_accuracy = []
    threshold = []

    for top_length in [0.5, 1, 2, 4]:

        idx_pred = argsorted_y_pred[-int(top_length*len(idx_true)):]

        topkl_accuracy += [np.size(np.intersect1d(idx_true, idx_pred)) \
                  / float(min(len(idx_pred), len(idx_true)))]
        threshold += [sorted_y_pred[-int(top_length*len(idx_true))]]

    auprc = average_precision_score(y_true, y_pred)

    print (("%.4f\t\033[91m%.4f\t\033[0m%.4f\t%.4f\t\033[94m%.4f\t\033[0m"
          + "%.4f\t%.4f\t%.4f\t%.4f\t%d") % (
          topkl_accuracy[0], topkl_accuracy[1], topkl_accuracy[2],
          topkl_accuracy[3], auprc, threshold[0], threshold[1],
          threshold[2], threshold[3], len(idx_true)))

In [8]:
#top k for splice sites similar to Splice AI?
print ("\n\033[1mDonor:\033[0m")
print_topl_statistics(np.asarray(Y_true_1),
                    np.asarray(Y_pred_1))

print ("\n\033[1mAcceptor:\033[0m")
print_topl_statistics(np.asarray(Y_true_2),
                          np.asarray(Y_pred_2))


Donor:
0.4938	0.4794	0.8375	0.9356	0.4476	0.4882	0.4723	0.1395	0.0071	9583

Acceptor:
0.4932	0.4847	0.8391	0.9336	0.4460	0.4899	0.4750	0.1400	0.0071	9583


## Binary RBP model evaluation

In [1]:
import tensorflow as tf
import h5py
import numpy as np
import rna_model
import torch
import h5py
from torch.utils.data import DataLoader
from tqdm import tqdm
import glob
import pytorch_lightning as pl
from torchinfo import summary
import sklearn.metrics as skl

Using TensorFlow backend.


In [2]:
data_dir = './data/RBP/i_2000_replicate_binary/i_2000_replicate_binary.h5'
test_loader = DataLoader(rna_model.h5dataset(data_dir,'test')
                    ,num_workers=4,pin_memory=True,batch_size = 64)
f = h5py.File(data_dir,'r')
y_test = f['y_test'][()]
print(y_test.shape)
f.close()

(2819, 120)


In [4]:
model_dir = glob.glob('/home/amber/multitask_RNA/wandb/run*m9rrwfw8/files/best_model.ckpt')[0]
model=rna_model.rbp_cnn(120,0.004)
model.load_state_dict(torch.load(model_dir)['state_dict'])

<All keys matched successfully>

In [5]:
pred_list = []
for i, batch in enumerate(tqdm(test_loader)):
        pred =model.predict_step(batch[0],i)
        pred_list.extend(pred.detach().numpy())
pred_list = np.array(pred_list)

100%|█████████████████████████████████████| 45/45 [00:10<00:00,  4.15it/s]


In [6]:
#Torch metrics
from torchmetrics import AUROC,AveragePrecision,Accuracy
accuracy = Accuracy()
print(accuracy(torch.tensor(pred_list),torch.tensor(y_test)))
auroc = AUROC(num_classes = 120)
print(auroc(torch.tensor(pred_list),torch.tensor(y_test)))
average_precision = AveragePrecision(num_classes=120)
print(average_precision(torch.tensor(pred_list),torch.tensor(y_test)))

tensor(0.9796)
tensor(0.7709)
tensor(0.2125)


/home/amber/tf_2/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
/home/amber/tf_2/lib/python3.8/site-packages/torchmetrics/functional/classification/average_precision.py:166: UserWarning: Average precision score for one or more classes was `nan`. Ignoring these classes in average
  warnings.warn(


In [7]:
pred_list_label = (pred_list > 0.5) 
print(skl.accuracy_score(y_test.flatten(),pred_list_label.flatten()))
auc = 0
class_i = 0
for i in range(120):
    try:
        auc += skl.roc_auc_score(y_test[:,i],pred_list[:,i])
        class_i += 1
    except:
        next
        
print(auc / class_i)
print(skl.average_precision_score(y_test,pred_list))

0.9795642662882819
0.8486948792437075
0.193024797354515


/home/amber/tf_2/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:877: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/amber/tf_2/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:877: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/amber/tf_2/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:877: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/amber/tf_2/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:877: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/amber/tf_2/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:877: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/amber/tf_2/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:877: Us

## Keras model with sklearn evaluation

In [7]:
import tensorflow as tf
from tensorflow import keras
import sklearn.metrics as skl
import glob
import h5py

In [8]:
model_dir = glob.glob('/home/amber/multitask_RNA/keras_script/wandb/run-*1pa6q71c/files/model-best.h5')[0]
model = tf.keras.models.load_model(model_dir,compile=False)

2022-06-16 23:13:20.422842: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-16 23:13:20.721686: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10418 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:c2:00.0, compute capability: 8.6


In [9]:
data_dir = './data/RBP/i_2000_replicate_binary/i_2000_replicate_binary.h5'
f = h5py.File(data_dir,'r')
y_test = f['y_test'][()]
x_test = f['x_test'][()]
print(y_test.shape)
print(x_test.shape)
f.close()

(2819, 120)
(2819, 2000, 4)


In [17]:
auroc = tf.keras.metrics.AUC(curve='ROC', name='auroc')
aupr = tf.keras.metrics.AUC(curve='PR', name='aupr')
model.compile(
            tf.keras.optimizers.Adam(learning_rate = 0.001),
             loss = tf.keras.losses.BinaryCrossentropy(from_logits=False, label_smoothing=0),
             metrics=['binary_accuracy',auroc,aupr]
             )

In [18]:
model.evaluate(x_test,y_test)

89/89 [==============================] - 1s 7ms/step - loss: 0.0657 - binary_accuracy: 0.9803 - auroc: 0.9197 - aupr: 0.4171


[0.06566571444272995,
 0.9802645444869995,
 0.9196731448173523,
 0.4171285033226013]

In [12]:
pred_test = model.predict(x_test)

In [16]:
pred_test_label = (pred_test > 0.5) 
print(skl.accuracy_score(y_test.flatten(),pred_test_label.flatten()))
auc = 0
class_i = 0
for i in range(120):
    try:
        auc += skl.roc_auc_score(y_test[:,i],pred_test[:,i])
        class_i += 1
    except:
        next
        
print(auc / class_i)
print(skl.average_precision_score(y_test,pred_test))

0.9802648693390091
0.8486160148422788
0.21685788253870014


/home/amber/tf_2/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:877: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/amber/tf_2/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:877: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/amber/tf_2/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:877: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/amber/tf_2/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:877: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/amber/tf_2/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:877: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/amber/tf_2/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:877: Us

In [10]:
pred_test_label.sum()

2433